# TP GAN
## Prérequis

Avoir suivi la formation GAN et avoir quelques bases avec Tensorflow (juste faire un modèle .Sequential). Vous pouvez vous mettre sous les yeux le TP CNN.

## Introduction

L'objectif de ce TP est de coder un GAN pour générer des petits chats trop mignons. Il abordera les notions principales vues pendant la formation GAN: les CNN, convolutions transposées, batchnorm, binary crossentropy, comment entraîner tout ce beau monde...etc (et peut-être les problèmes de GAN si ça arrive mais pas de panique).

La plupart des détails sont déjà implémentés, le plus important étant d'élaborer l'architecture du générateur et du discriminateur ainsi que de comprendre comment les entraîner.


## Les imports

On va d'abord importer les librairies nécessaires dont les classiques numpy, matplotlib ainsi que tensorflow, keras et certaines classes utilisés assez souvent (layers, models ...etc)

In [ ]:
from tensorflow import keras
from keras import models,layers
import matplotlib.pyplot as plt
import tensorflow as tf
import time
from IPython.display import clear_output
import gdown
import os
from tqdm import tqdm

## Téléchargement du dataset

Importons maintenant le dataset. On va télécharget le dataset de chats depuis ce [drive](https://drive.google.com/uc?id=1F9I7iDmQ_I9Qsrav1UXlD4OiIBVSU5sl) avec la commande gdown (ou à la mano si vous préférez).


In [ ]:
#Téléchargement du dataset
url = 'https://drive.google.com/uc?id=1F9I7iDmQ_I9Qsrav1UXlD4OiIBVSU5sl'
output = 'dataset.tgz'
if not os.path.exists(output):
    gdown.download(url, output, quiet=False)

#Dézippage du dataset

def unzip(zip_file, dest_dir):
    import zipfile
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(dest_dir)
        
unzip('dataset.tgz', './')

## Quelques paramètres généraux

Les paramètres peuvent être modifiés pour tester un peu (sauf peut-être la taille de l'image pour ce dataset).

Pour ce dataset, on peut prendre une batch size pas trop grande sinon Colab crash (ou votre PC). De même, la dimension de l'espace latent peut être ajustée, ici vu que les chats c'est pas si simple, environ 100 c'est bien.

Pour charger le dataset, on utilise la méthode `image_dataset_from_directory` qui comme son nom l'indique prend juste le path du dossier d'images et en fait un dataset tensorflow que l'on peut mélanger, batcher...etc

Les images ont leurs pixels entre 0 et 255 qu'on va renormaliser entre -1 et 1, ce qui est plus adapté pour les réseaux de neurones et bien pour des GAN car on a une moyenne nulle.
On va aussi les afficher parce que c'est bien de savoir sur quoi on travaille quand même.


In [ ]:
BATCH_SIZE = 64
LATENT_DIM = 100
IMG_SHAPE = (64,64,3)

x_train = keras.utils.image_dataset_from_directory('dataset',image_size=(64,64),batch_size=BATCH_SIZE,shuffle=True,seed=123,labels=None)

#Normalisation des données
x_train = x_train.map(lambda x: (x-127.5)/127.5)


In [ ]:
test_batch = next(iter(x_train))
fig = plt.figure(figsize=(12,12))
for i in range(25):
  plt.subplot(5,5,i+1)
  plt.axis('off')
  #On oublie pas de faire image * 0.5 + 0.5 pour revenir dans [0,1]
  plt.imshow(test_batch[i]*0.5+0.5)
plt.show()

## Le discriminateur

On va faire le discriminateur.

En entrée : la shape de l'image, typiquement (64,64,3) ici (c'est couleur pour rappel donc il y a bien 3 canaux).

Pour vous aider, un petit rappel des blocs à mettre:
- `Conv2D(filtres,kernel_size,strides,padding='same' ou 'valid')`

  Typiquement, on utilise une taille de kernel de 3 et des filtres qui sont des puissances de 2 (32,64,128...).
  
  Pour rappel, le padding `'same'` équivaut à rajouter des zéros pour garder la même taille d'image en sortie (si les strides sont de 1) et avec `'valid'` on en rajoute pas. `'same'` est donc conseillé ici pour éviter les surprises (et de se torturer la tête sur la taille en sortie même si vous savez évidemment que c'est $\frac{n+2p-f}{s}+1$).
   
   On ne met pas l'activation tout de suite car on ajoute d'abord de la batch normalization pour renormaliser le batch en 0 et ainsi profiter du comportement de la ReLU en ce point.

  Pour rappel, on augmente le nombre de filtres au fur et à mesure de l'architecture dans le discriminateur.
- `BatchNormalization()` (pas d'arguments, on laisse par défaut)
- `LeakyReLU(alpha=0.2)` (pente de la leaky relu dans $]-\infty,0]$)

Mettez 3-4 blocs comme ça et faites des essais.

Ici pas de `Pooling` car on réduit la taille des images directement avec du stride (typiquement 2 à chaque Convolution en comptant bien la taille qu'on obtient à la fin). 

En sortie on veut une dimension (1) grâce à une `Flatten` puis des `Dense` en oubliant pas la sigmoïde `activation='sigmoid'`à la fin. 


*Alternative*:
- C'est aussi possible de bien calculer la taille de l'image pour terminer par une `Convolution` avec une sortie de dimension (1,1,1) puis une `Flatten` (un peu mieux).

  **Exemple** : 
  - Image de (64,64,3) -> 4 Blocs de convolution avec du stride de 2 : $64/2^4$ -> Feature map de taille (4,4,nombre_de_filtres). 

    On peut ensuite finir avec une `Conv2D` avec une taille de filtre de 4 et du padding `'valid'` pour juste faire une combinaison de tous les pixels restants, ce qui nous donne bien notre unique pixel de dimension (1,1,1) sans oublier la sigmoïde et on peut ensuite `Flatten` tout ça).

Pas besoin de compiler car on va faire un entraînement personnalisé après.

<details> 
<summary>Antisèche</summary>
Conv2D(32 filtres,kernel taille 3,stride 2,padding same) -> BN -> LR <br />  
-> Conv2D(64,kernel 3,stride 2,padding same) -> BN -> LR <br />  
-> Conv2D(128,3,2,same) -> BN -> LR <br />  
-> Conv2D(256,3,2,same) -> BN -> LR <br />  
-> Conv2D(1,4,1,valid) -> Sigmoïde -> Flatten ou Flatten -> Dense(1) -> Sigmoïde
</details>

In [ ]:
def define_discriminator(im_shape=(64,64,3)):
  model = models.Sequential()
  ###Ajouter les couches ici (rappel: on peut utiliser model.add(layers.CoucheX(arguments)))
  model.add(layers.Conv2D(64,3,2,padding='same',input_shape=im_shape))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(alpha=0.2))
  model.add(layers.Conv2D(128,3,2,padding='same'))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(alpha=0.2))
  model.add(layers.Conv2D(256,3,2,padding='same'))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(alpha=0.2))
  model.add(layers.Conv2D(512,3,2,padding='same'))
  model.add(layers.BatchNormalization())
  model.add(layers.LeakyReLU(alpha=0.2))
  model.add(layers.Conv2D(1,4,1,padding='valid',activation='sigmoid'))
  model.add(layers.Flatten())

  return model

##Petit tips : faire un .summary() pour vérifier qu'on s'est pas trompé dans les dimensions de sortie
define_discriminator().summary()

## Le générateur

On va faire le générateur.
En entrée : un vecteur de l'espace latent de dimension `latent_dim`.

`Dense(4 x 4 x 1024,input_shape=(latent_dim, ))`

`Reshape((4,4,1024))`


On peut aussi directement redimensionner le vecteur de l'espace latent sans la couche Dense.


Pour vous aider, un petit rappel des blocs à mettre ensuite:
- `Conv2DTranspose(filtres,kernel_size,strides,padding='same' ou 'valid')`

  Pour rappel, on diminue le nombre de filtres au fur et à mesure de l'architecture dans le générateur (512,256,...)

- `BatchNormalization()`
- `ReLU()`

Mettez 2-3 blocs comme ça et faites des essais aussi. Faites bien attention à la taille de vos images tout au long de l'architecture pour bien avoir la taille d'image finale voulue.

**Exemple**:

- On a transformé notre vecteur latent en image de dimension (4,4,1024). Pour avoir du (64,64,3), il faut donc 4 blocs de `Conv2DTranspose` (avec padding) pour avoir $4*2^4=64$. Libre à vous de changer l'entrée pour mettre le nombre de blocs que vous voulez.


Ici aussi pas de `UpSampling2D` car on augmente la taille avec du stride (2 aussi souvent) à chaque convolution transposée.

En sortie on veut une dimension (64,64,3) grâce à une `Conv2D` avec 3 filtres, on oublie pas de prendre une activation en tangente hyperbolique `'tanh'` pour avoir des pixels dans [-1,1].

<details>
<summary>Antisèche</summary>
  Dense(4*4*1024 neurones) -> Reshape(en (4,4,1024))
  -> ConvTransposée(256 filtres,kernel taille 3,stride 2,padding same) -> BN -> ReLU <br />  
  -> ConvTransposée(128,kernel 3,stride 2,padding same) -> BN -> ReLU <br />  
  -> ConvTransposée(64,3,2,same) -> BN -> ReLU <br />  
  -> ConvTransposée(32,3,2,same) -> BN -> ReLU <br />  
  -> ConvTransposée(3,3,1,same) -> Tanh
</details>

In [ ]:
def define_generator(latent_dim=LATENT_DIM):
	model = models.Sequential()
	###Ajouter les couches ici
	model.add(layers.Dense(4*4*1024,input_shape=(latent_dim,)))
	model.add(layers.Reshape((4,4,1024)))
	model.add(layers.Conv2DTranspose(256,3,2,padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.Conv2DTranspose(128,3,2,padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.Conv2DTranspose(64,3,2,padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.Conv2DTranspose(32,3,2,padding='same'))
	model.add(layers.BatchNormalization())
	model.add(layers.ReLU())
	model.add(layers.Conv2D(3,3,1,padding='same',activation='tanh'))

	return model

##Petit tips : faire un .summary() pour vérifier qu'on s'est pas trompé dans les dimensions de sortie
define_generator().summary()

## Le train step

Cette partie est assez importante car elle permet de comprendre comment on entraîne vraiment un GAN, c'est-à-dire à quoi on compare les sorties pour entraîner correctement le discriminateur et le générateur. Ici, pas de `.fit` malheureusement.
On va définir un `train_step`, c'est à dire ce qu'on va faire comme opérations à chaque batch :

Entraîner le discriminateur :

- Générer des images fausses à partir de bruit gaussien et en prédire les labels : on a besoin ici d'un vecteur latent de dimension `(batch_size,latent_dim)`. Ensuite, on fait passer ce bruit dans le générateur pour obtenir des fausses images. Enfin, on récupère la sortie du discriminateur sur celles-ci.


```
my_latent_vector = tf.random.normal(shape)
fake_images = generator(...)
fake_predictions = discriminator(...)
```


- Prendre des images vraies du dataset et en prédire aussi les labels. Donc la sortie du discriminateur sur les vraies images.


```
real_predictions = ...
```


- Calculer la loss en comparant les prédictions sur les fausses avec des 0 et les prédictions sur les vraies avec des 1.



La binary crossentropy prend en argument les labels visés puis ceux prédits.
Pour avoir des 1 ou des 0: `tf.ones(shape)` ou `tf.zeros(shape)` avec shape = (batch_size,1). Dans l'exemple suivant, cela corresponds aux `y_true` et les prédictions précédentes aux `y_pred`.
```
discriminator_loss_on_real = loss(y_true1,y_pred1)
discriminator_loss_on_fake = loss(y_true2,y_pred2)
discriminator_loss = discriminator_loss_on_real + discriminator_loss_on_fake
```


- Calculer les gradients en fonction de la loss calculée et les différents paramètres du modèle
- On applique les gradients calculés avec l'optimisateur choisi

Entraîner le générateur (presque la même chose):
- Générer des images fausses à partir de bruit gaussien et en prédire les labels.
- Calculer la loss en comparant les prédictions sur les fausses avec des 1 (on veut tromper le discriminateur).
- Calculer les gradients en fonction de la loss calculée et les différents paramètres du modèle
- On applique les gradients calculés avec l'optimisateur choisi

In [ ]:
def train_step(real_images,generator,discriminator,loss,g_opt,d_opt):
  batch_size = tf.shape(real_images)[0]
  global LATENT_DIM
  with tf.GradientTape() as disc_tape:

    ###A compléter###
    latent_vector = tf.random.normal(shape=(batch_size,LATENT_DIM))
    fake_images = generator(latent_vector)
    #Ce sont les prédictions du discriminateur
    real_predictions = discriminator(real_images)
    fake_predictions = discriminator(fake_images)

    #Les labels sont les vrais labels des images du dataset (0 ou 1)
    real_labels = tf.ones(shape=(batch_size,1))
    fake_labels = tf.zeros(shape=(batch_size,1))

    disc_loss_on_real = loss(real_labels,real_predictions)
    disc_loss_on_fake = loss(fake_labels,fake_predictions)
    disc_loss = disc_loss_on_real + disc_loss_on_fake
    ######

  disc_grad = disc_tape.gradient(disc_loss,discriminator.trainable_variables)
  d_opt.apply_gradients(zip(disc_grad,discriminator.trainable_variables))
  
  with tf.GradientTape() as gen_tape:

    ###A compléter###
    latent_vector = tf.random.normal(shape=(batch_size,LATENT_DIM))
    generated_images = generator(latent_vector)
    fake_predictions = discriminator(generated_images)

    #Rappel : on veut comparer les images générées à des 1 pour tromper le discriminateur cette fois
    real_labels = tf.ones(shape=(batch_size,1))
    gen_loss = loss(real_labels,fake_predictions)
    ######

  gen_grad = gen_tape.gradient(gen_loss,generator.trainable_variables)
  g_opt.apply_gradients(zip(gen_grad,generator.trainable_variables))
  return gen_loss,disc_loss

## Le train
On code la fonction d'entraînement principale.
Il reste à compléter la loss et les optimisateurs à utiliser, typiquement ici la `BinaryCrossentropy` et `Adam(learning_rate=2e-4,beta_1=0.5)`.

Dans un premier temps, on peut prendre les mêmes optimiseurs pour les deux quitte à adapter pour tester après (changer le learning rate par exemple pour rééquilibrer un peu l'entraînement).

Il aurait été possible de faire une boucle d'entraînement plus poussée ou d'utiliser d'autres méthodes de keras directement (suivre les tutoriels sur https://keras.io/guides/)

In [ ]:
def train(dataset,generator,discriminator,epochs,fixed_seed = tf.random.normal((25,LATENT_DIM)),seed=42):
  
  ###A compléter###
  loss = keras.losses.BinaryCrossentropy()
  g_opt = keras.optimizers.Adam(learning_rate=2e-4,beta_1=0.5)
  d_opt = keras.optimizers.Adam(learning_rate=2e-4,beta_1=0.5)

  Lgen_loss = []
  Ldisc_loss = []
  X = []
  j = 0
  ######

  for epoch in range(epochs):
    progress_bar = tqdm(dataset)
    ##Vu que c'est un dataset tensorflow, on ne peut itérer directement dessus avec son indice. On va juste prendre à chaque fois batch par batch.
    for _,image_batch in enumerate(progress_bar):
        j += 1
        gen_loss, disc_loss = train_step(image_batch,generator,discriminator,loss,g_opt,d_opt)
        
        X.append(j)
        Lgen_loss.append(gen_loss)
        Ldisc_loss.append(disc_loss)

    clear_output(wait=False)
    generate_and_save_plots(X, Lgen_loss, Ldisc_loss,  epoch+1)
    summarize_performance(generator,fixed_seed)

## L'affichage à chaque epoch

Petite fonction qui affiche les images obtenues à chaque epoch. On va **afficher** 25 images avec la même seed (toujours du même vecteur latent) pour voir l'amélioration progressive de l'image. Ce n'est pas de l'overfitting sur un seul vecteur car on **entraîne** bien à partir de vecteurs différents à chaque fois avant.

On affiche aussi après chaque epoch les courbes des loss du générateur et du discriminateur pour suivre l'entraînement (empêcher des vanishing gradients ou des exploding gradients).

In [ ]:
def summarize_performance(generator,fixed_seed):
  fake_images = generator.predict(fixed_seed)
  fig = plt.figure(figsize=(12,12))
  for i in range(25):
    plt.subplot(5,5,i+1)
    plt.axis('off')
    plt.imshow(fake_images[i]*0.5+0.5)
  plt.show()

In [ ]:
def generate_and_save_plots(X, Lgen_loss, Ldisc_loss, epoch):
    fig = plt.figure(figsize=(4,4))
    plt.plot(X,Lgen_loss, label = 'gen_loss')
    plt.plot(X,Ldisc_loss, label = 'disc_loss')
    plt.legend()
    plt.show()

## Ici vous lancez tout!
Choisissez le nombre d'epochs que vous voulez. Pour MNIST, quelques dizaines d'epochs sont suffisantes pour avoir des résultats mais vous pourrez toujours relancer si vous n'êtes pas satisfaits.

In [ ]:
generator = define_generator(latent_dim=LATENT_DIM)
discriminator = define_discriminator(im_shape=(64,64,3))
EPOCHS = 10
train(x_train,generator,discriminator,EPOCHS)

# A développer
## Autres datasets

Vous pouvez essayer de générer des images sur d'autres datasets comme Fashion MNIST, CIFAR, CelebA ou n'importe quel type d'images qui vous font plaisir. S'il faut le télécharger et l'importer sur Colab, vous pouvez directement monter votre Drive et uploader votre dataset sur ce dernier : 


```
from google.colab import drive
drive.mount('/content/drive/')
```

## Autres architectures

Vous pouvez rajouter des blocs dans le générateur ou le discriminateur, essayer d'ajouter du Dropout, enlever les biais dans les couches, modifier la dimension de l'espace latent, initialiser les poids d'une certaine façon...etc Si vous êtes déjà à jour sur les CNN avancés (ResNET, MobileNet, EfficientNet), vous pouvez essayer de faire un GAN sur ces bases, ce qui permettra de résoudre certains problèmes comme les vanishing gradient.

## Tenter de forcer des problèmes

Faites n'importe quoi ! 

Plus sérieusement, cela peut arriver surtout sur des datasets plus compliqués, la plupart des choses fonctionnent pour MNIST à part si on fait exprès de provoquer des problèmes.

Vous pouvez essayer par exemple de déséquilibrer l'entraînement en modifiant les learning rate, en rajoutant beaucoup de couches seulement d'un côté...etc

Ici on peut forcer le mode collapse en prenant une dimension latente de seulement 1. Le générateur aura alors plus de risques de générer des images sur un seul mode.

## Tips d'entraînement possibles

- Utiliser une loss différente, du genre la Wasserstein loss.

- Changer le label visé pour les images vraies de 1 en 0.9

- Rajouter du bruit sur les images

- Ajouter des labels à l'entrée du générateur et du discriminateur (vous pourrez alors même choisir les classes des images générées)

- Entraîner le discrminateur plus que le générateur (typiquement entre 3 à 5 boucles d'entraînement à chaque fois que le générateur en fait une).


# Ce qu'il faut retenir de ce TP
- Architecture du générateur et du discriminateur :

  `Conv2D(Transpose) -> BatchNorm -> ReLU ou LeakyReLU`

  +Spécifités en entrée et sortie selon ce que l'on veut

  Eviter les fully connected/denses si possible

- Faire une boucle d'entraînement from scratch avec un train_step (très utile pour faire des choses plus compliquées)

  `Loss -> gradient_tape.gradient() -> optimizer.apply_gradients`